In [41]:
#test out the Adjustments Prediction code for XLWings Plugin

#Request from Excel
import json
jsonReq='{"AdjustedField":"RiskCategory","AdjustmentName":"Noise_116z","CustomerID":"CUST981372","CustomerType":"SME","AccountType":"Current","Approach":"ADV","RiskCategory":"Medium","SubDivision":"HSBC_DEBTS","IsDefaulted":1,"CountryCode":"GB","Drawn":742441,"Undrawn":934585,"PD":0.14}'
request = json.loads(jsonReq)



In [49]:
#Load in the model
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

#Load our model
modelPath='Models/AdjustmentValues/model.h5'
with tf.device('/gpu:0'):
    model = keras.models.load_model(modelPath)

#Load column names
column_names=[]
import json
with open('Models/AdjustmentValues/in_columnnames.json') as json_file:
    column_names = json.load(json_file)

#Load numeric fields definition
numeric_fields=[]
import json
with open('Models/AdjustmentValues/in_numericfield.json') as json_file:
    numeric_fields = json.load(json_file)

#Load field metrics for scaling
field_metrics={}
import json
with open('Models/AdjustmentValues/in_fieldmetrics.json') as json_file:
    field_metrics = json.load(json_file)

#Load softmaxlkp
softmaxlkp={}
import json
with open('Models/AdjustmentValues/softmaxlkp.json') as json_file:
    softmaxlkp = json.load(json_file)

#Build a pandas dataframe filled with zeros
df = pd.DataFrame(0, index=np.arange(1), columns=column_names)
df




,Drawn,Undrawn,PD,IsDefaulted,AdjustmentType_Amend,AdjustmentName_BREXIT,AdjustmentName_BREXIT_DEFAULTED,AdjustmentName_Defaulting Cleanup,AdjustmentName_German Retail Downturn,AdjustmentName_Market Downturn,...,RiskCategory_Medium,SubDivision_HSBC_AP,SubDivision_HSBC_CORP,SubDivision_HSBC_DEBTS,SubDivision_HSBC_IR,SubDivision_HSBC_UK,SubDivision_HSBC_US,CountryCode_DE,CountryCode_GB,CountryCode_US
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
#Populate the dataframe in AI format with numeric values
def Normalize(col, min, max):
    return (col-min)/(max-min)*2-1

def ApplyOneHotField(df,fieldName,valu):
    fieldNameOH=fieldName+'_'+str(valu)

    #Check if the field exists
    if fieldNameOH in df.columns:
        #If it does, set it to 1
        df[fieldNameOH]=1
    else:
        print("***No field found for "+fieldNameOH+" in dataframe with value "+str(valu)+" - skipping")

    return df

#Load and Normalize all numeric fields
for field in numeric_fields:
    #Load un-normalized value
    df[field]=request[field]

    if field in field_metrics:
        #Load metrics
        min=field_metrics[field][0]
        max=field_metrics[field][1]

        #Normalize
        df[field]=Normalize(df[field], min, max)

#Apply the strings in the rest of request as onehot encoded
for field,valu in request.items():
    if field in numeric_fields:
        #Skip 
        continue
    else:
        df=ApplyOneHotField(df,field,valu)       

#drop index
df=df.drop(df.columns[0], axis=1)

df

,Undrawn,PD,IsDefaulted,AdjustmentType_Amend,AdjustmentName_BREXIT,AdjustmentName_BREXIT_DEFAULTED,AdjustmentName_Defaulting Cleanup,AdjustmentName_German Retail Downturn,AdjustmentName_Market Downturn,AdjustmentName_Noise_116z,...,RiskCategory_Medium,SubDivision_HSBC_AP,SubDivision_HSBC_CORP,SubDivision_HSBC_DEBTS,SubDivision_HSBC_IR,SubDivision_HSBC_UK,SubDivision_HSBC_US,CountryCode_DE,CountryCode_GB,CountryCode_US
0,0.869216,-0.911111,1,0,0,0,0,0,0,1,...,1,0,0,1,0,0,0,0,1,0


In [53]:
#Make a prediction

with tf.device('/gpu:0'):
    predictions = model.predict(df)
    print(predictions)

#Get argmax value
argmax = np.argmax(predictions[0])    
print(softmaxlkp[str(argmax)])

1/1 [==============================] - 0s 13ms/step
[[3.9629409e-05 2.9879914e-06 2.9231107e-04 7.9300393e-05 3.4875171e-05
  7.0658207e-05 2.0016045e-05 2.5696550e-05 4.4312328e-05 3.0500745e-05
  1.0127325e-04 2.1600839e-05 1.9765544e-05 5.1700703e-05 3.4596589e-05
  7.1608803e-05 8.2676641e-05 2.3111115e-05 4.6758421e-05 8.4985186e-05
  9.3227602e-05 4.9200029e-05 2.8678740e-05 2.2447502e-04 1.9441715e-04
  6.4026848e-05 1.1565404e-04 2.1106525e-01 4.0951595e-04 1.6096704e-04
  4.9168655e-05 7.6571375e-02 6.0716647e-01 7.5798431e-05 2.3261296e-04
  2.3627534e-05 7.6112927e-05 5.0520845e-05 2.6896480e-04 3.1300588e-04
  1.5763214e-04]]
Medium
